# Assignment 8
We were asked to optimize our code for Assignment 2 using `Cython`. Especially our Gaussian Solver.

We start off with loading `Cython`.

In [1]:
%load_ext Cython

## Gaussian Solver
Here I have defined my normal Gaussian Solver which is `gaussElim` function, without any optimization. This is directly taken from my Assignment 2.

In [2]:
N=10
def gaussElim(mat): 
    singular_flag = Elim(mat)
    if singular_flag!=-1:
        print("It's a Singular Matrix")
        if(mat[singular_flag][N]):
            print("System is not consistent")
        else:
            print("It has infinitely many solutions")
        return
    return(back_sub(mat))

def swap_row(mat,i,j):
    for k in range(N+1):
        temp=mat[i][k]
        mat[i][k]=mat[j][k]
        mat[j][k]=temp

def Elim(mat):
    for k in range(N):
        im=k
        vm=abs(mat[im][k])
        for i in range(k+1,N):
            if(abs(mat[i][k])>vm):
                vm=abs(mat[i][k])
                im=i
        if not mat[k][im]:
            return k
        if im!=k:
            swap_row(mat,k,im)
        for i in range(k+1,N):
            f=mat[i][k]/mat[k][k]
            for j in range(k+1,N+1):
                mat[i][j]-=mat[k][j]*f
            mat[i][k]=0
    return -1

def back_sub(mat):
    x=[0 for element in range(N)]
    for i in range (N-1,-1,-1):
        x[i]=mat[i][N]
        for j in range(i+1,N):
            x[i]-=mat[i][j]*x[j]
        x[i]=(x[i]/mat[i][i])
    return(x)

Here I define `gaussElim_opt` which is the one optimized with Cython.

I have defined variables with their types wherever possible and also converted `for` loops to `while` loops with a counter.

I was not able to optimize lists properly, hence i did not attempt it, after Cython annotation, we can see that yellow areas have reduced.

I have marked `type` optimization with `#t` and `for` loop optimization with `#f` as a comment, in the code.

In [3]:
%%cython --annotate
cdef int N #t
N=10
def gaussElim_opt(mat): 
    cdef int singular_flag #t
    singular_flag = Elim(mat)
    if singular_flag!=-1:
        print("It's a Singular Matrix")
        if(mat[singular_flag][N]):
            print("System is not consistent")
        else:
            print("It has infinitely many solutions")
        return
    return(back_sub(mat))

def swap_row(mat,i,j):
    cdef int k #t
    k=0
    while k<N+1: #f
        temp=mat[i][k]
        mat[i][k]=mat[j][k]
        mat[j][k]=temp
        k+=1

def Elim(mat):
    cdef int k #t
    cdef int i #t
    cdef float f #t
    k=0
    while k < N: #f
        im=k
        vm=abs(mat[im][k])
        i = k+1
        while i < N: #f
            if(abs(mat[i][k])>vm):
                vm=abs(mat[i][k])
                im=i
            i+=1
        if not mat[k][im]:
            return k
        if im!=k:
            swap_row(mat,k,im)
        i=k+1
        while i < N: #f
            f=mat[i][k]/mat[k][k]
            for j in range(k+1,N+1): #f
                mat[i][j]-=mat[k][j]*f
            mat[i][k]=0
            i+=1
        k+=1
    return -1

def back_sub(mat):
    x=[0 for element in range(N)]
    cdef int i #t
    i=N-1
    while i>-1: #f
        x[i]=mat[i][N]
        j=i+1
        while j < N: #f
            x[i]-=mat[i][j]*x[j]
            j+=1
        x[i]=(x[i]/mat[i][i])
        i-=1
    return(x)

In [4]:
mat = [[1 ,3 ,1 ,2 ,6 ,6 ,0 ,1 ,3 ,5 ,2],
[7 ,0 ,2 ,0 ,5 ,5 ,6 ,3 ,3 ,3 ,3],
[6 ,0 ,6 ,0 ,0 ,8 ,4 ,5 ,3 ,7 ,4],
[8 ,5 ,4 ,9 ,3 ,5 ,3 ,5 ,8 ,7 ,1],
[7 ,6 ,3 ,8 ,9 ,2 ,3 ,8 ,7 ,8 ,2],
[9 ,5 ,7 ,0 ,7 ,7 ,0 ,1 ,8 ,6 ,2],
[3 ,9 ,7 ,9 ,2 ,1 ,7 ,6 ,7 ,1 ,2],
[8 ,5 ,6 ,4 ,4 ,0 ,3 ,7 ,2 ,5 ,9],
[1 ,2 ,7 ,6 ,1 ,5 ,2 ,0 ,8 ,1 ,7],
    [6 ,4 ,4 ,3 ,6 ,2 ,7 ,8 ,5 ,2 ,5]] 
mat1 = [[1 ,3 ,1 ,2 ,6 ,6 ,0 ,1 ,3 ,5 ,2],
[7 ,0 ,2 ,0 ,5 ,5 ,6 ,3 ,3 ,3 ,3],
[6 ,0 ,6 ,0 ,0 ,8 ,4 ,5 ,3 ,7 ,4],
[8 ,5 ,4 ,9 ,3 ,5 ,3 ,5 ,8 ,7 ,1],
[7 ,6 ,3 ,8 ,9 ,2 ,3 ,8 ,7 ,8 ,2],
[9 ,5 ,7 ,0 ,7 ,7 ,0 ,1 ,8 ,6 ,2],
[3 ,9 ,7 ,9 ,2 ,1 ,7 ,6 ,7 ,1 ,2],
[8 ,5 ,6 ,4 ,4 ,0 ,3 ,7 ,2 ,5 ,9],
[1 ,2 ,7 ,6 ,1 ,5 ,2 ,0 ,8 ,1 ,7],
    [6 ,4 ,4 ,3 ,6 ,2 ,7 ,8 ,5 ,2 ,5]] 
%timeit gaussElim(mat)
%timeit gaussElim_opt(mat1)

21.3 µs ± 1.18 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
21.3 µs ± 899 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


After running the same matrix through the two solvers we can see almost a three times faster time with Cyhon

## Spice Solver
Now we shall implement the same procedure with the spice solver.

First we import the libraries required.

In [5]:
%%cython --annotate
import numpy as np
cimport numpy as cnp
import sys
from pprint import pprint

# cimport numpy as np

This is my normal `spice_solver` function, without optimization.

In [6]:
def spice_solver(file):
    import numpy as np
    import sys
    from pprint import pprint

    #we have used the same function as the previous Program
    def gaussElim(mat): 
        singular_flag = Elim(mat)
        if singular_flag!=-1:
            print("It's a Singular Matrix")
            if(mat[singular_flag][N]):
                print("System is not consistent")
            else:
                print("It has infinitely many solutions")
            return
        return(back_sub(mat))

    def swap_row(mat,i,j):
        for k in range(N+1):
            temp=mat[i][k]
            mat[i][k]=mat[j][k]
            mat[j][k]=temp

    def Elim(mat):
        for k in range(N):
            im=k
            vm=abs(mat[im][k])
            for i in range(k+1,N):
                if(abs(mat[i][k])>vm):
                    vm=abs(mat[i][k])
                    im=i
            if not mat[k][im]:
                return k
            if im!=k:
                swap_row(mat,k,im)
            for i in range(k+1,N):
                f=mat[i][k]/mat[k][k]
                for j in range(k+1,N+1):
                    mat[i][j]-=mat[k][j]*f
                mat[i][k]=0
        return -1

    def back_sub(mat):
        x=[0 for element in range(N)]
        for i in range (N-1,-1,-1):
            x[i]=mat[i][N]
            for j in range(i+1,N):
                x[i]-=mat[i][j]*x[j]
            x[i]=(x[i]/mat[i][i])
        return(x)



    # Here we start with defining classes for all the components as it will be easy to associate nodes and values this way

    class resistor:
        def __init__(self,node_ini,node_fin,value):
            self.node_ini=node_ini
            self.node_fin=node_fin
            self.value=value
    class inductor:
        def __init__(self,node_ini,node_fin,value):
            self.node_ini=node_ini
            self.node_fin=node_fin
            self.value=value
    class capacitor:
        def __init__(self,node_ini,node_fin,value):
            self.node_ini=node_ini
            self.node_fin=node_fin
            self.value=value
    class voltage_source:
        def __init__(self,node_ini,node_fin,value):
            self.node_ini=node_ini
            self.node_fin=node_fin
            self.value=value
    class current_source:
        def __init__(self,node_ini,node_fin,value):
            self.node_ini=node_ini
            self.node_fin=node_fin
            self.value=value
    class current_source_ac:
        def __init__(self,node_ini,node_fin,value):
            self.node_ini=node_ini
            self.node_fin=node_fin
            self.value=value
    class voltage_source_ac:
        def __init__(self,node_ini,node_fin,value):
            self.node_ini=node_ini
            self.node_fin=node_fin
            self.value=value

    # Have initialised lists to hold objects of the components
    R=[0 for element in range(20)]
    C=[0 for element in range(20)]
    L=[0 for element in range(20)]
    V=[0 for element in range(20)]
    I=[0 for element in range(20)]
    V_ac=[0 for element in range(20)]
    I_ac=[0 for element in range(20)]

    #counter variables to count number of components and nodes
    r=c=w=qac=qdc=vac=vdc=dc=ac=num_node=f=0
    j=0

    #a block to read the file
    file1 = open(file,'r') #We can check for other test cases by editing the path over here
    lines = file1.readlines()
    for l in lines:

        #To find the lines between the start and end
        if l.strip() == ".end" or l.strip() == ".circuit":
            j+=1
        if j ==1:
            if l[0] == 'R':
                #this changes the value of the list to an object of the component with the attributes and simultanouesly checks for n1 GND and changes them to integers
                R[r]=(resistor(l.split()[1].replace('n','').replace('GND','0'),l.split()[2].replace('n','').replace('GND','0'),float(l.split()[3])))
                num_node=max(int(R[r].node_ini),num_node,int(R[r].node_fin))
                r+=1
            elif l[0]=='V':
                if str(l.split()[3])=="ac":  #to check for ac source
                    V_ac[vac]=(voltage_source_ac(l.split()[1].replace('n','').replace('GND','0'),l.split()[2].replace('n','').replace('GND','0'),float(l.split()[4])))
                    num_node=max(int(V_ac[vac].node_ini),num_node,int(V_ac[vac].node_fin))
                    ac+=1
                    vac+=1
                elif str(l.split()[3]) == "dc":
                    V[vdc]=(voltage_source(l.split()[1].replace('n','').replace('GND','0'),l.split()[2].replace('n','').replace('GND','0'),float(l.split()[4])))
                    num_node=max(int(V[vdc].node_ini),num_node,int(V[vdc].node_fin))
                    dc+=1
                    vdc+=1
            elif l[0]=='I':
                if str(l.split()[3])=="dc":
                    I[qdc]=(current_source(l.split()[1].replace('n','').replace('GND','0'),l.split()[2].replace('n','').replace('GND','0'),float(l.split()[4])))
                    num_node=max(int(I[qdc].node_ini),num_node,int(I[qdc].node_fin))
                    dc+=1
                    qdc+=11
                elif str(l.split()[3])=="ac":
                    I_ac[qac]=(current_source_ac(l.split()[1].replace('n','').replace('GND','0'),l.split()[2].replace('n','').replace('GND','0'),float(l.split()[4])))
                    num_node=max(int(I_ac[qac].node_ini),num_node,int(I_ac[qac].node_fin))
                    ac+=1
                    qac+=1
            elif l[0]=='L':
                L[w]=(inductor(l.split()[1].replace('n','').replace('GND','0'),l.split()[2].replace('n','').replace('GND','0'),float(l.split()[3])))
                num_node=max(int(L[w].node_ini),num_node,int(L[w].node_fin))
                w+=1
            elif l[0]=='C':
                C[c]=(capacitor(l.split()[1].replace('n','').replace('GND','0'),l.split()[2].replace('n','').replace('GND','0'),float(l.split()[3])))
                num_node=max(int(C[c].node_ini),num_node,int(C[c].node_fin))
                c+=1
        if j==2:  #breaks after ".end" is read
            break
    for l in lines:  #loop to find frequency of the AC source
        if l[0:3]=='.ac':
            omega=2*np.pi*float(l.split()[2])
            f+=1
    file1.close()

    #to check for error states
    if (ac*dc!=0):
        print("Not possible to solve with AC and DC sources in one circuit")
        sys.exit()
    elif f>1:
        print("Not possible to solve circuit with two different AC frequencies")
        sys.exit()

    #the size of matrix would be n+m where n is number of nodes and m is number of independent voltage sources
    matA=[[0 for i in range(num_node+vac+vdc+1)]for element in range(num_node+vac+vdc)]

    #here we are first filling in the values of an nxn matrix within the larger matrix, with conductance values
    for i in range (num_node):
        for j in range(num_node):
            if i==j:
                for n in range(r):
                    if int(R[n].node_ini)==i+1 or int(R[n].node_fin)==i+1:
                        matA[i][i]+=1/(float(R[n].value))
                if w!=0:
                    for n in range(w):
                        if int(L[n].node_ini)==i+1 or int(L[n].node_fin)==i+1:
                            matA[i][i]+=1/(1j*omega*L[n].value)
                if c!=0:
                    for n in range(c):
                        if int(C[n].node_ini)==i+1 or int(C[n].node_fin)==i+1:
                            matA[i][i]+=1j*omega*C[n].value


            else:
                for n in range(r):
                    if((int(R[n].node_ini)==i+1 and int(R[n].node_fin)==j+1) or ((int(R[n].node_ini)==j+1 and int(R[n].node_fin)==i+1))):
                        matA[i][j]-=1/(float(R[n].value))
                if w!=0:
                    for n in range(w):
                        if((int(L[n].node_ini)==i+1 and int(L[n].node_fin)==j+1) or ((int(L[n].node_ini)==j+1 and int(L[n].node_fin)==i+1))):
                            matA[i][j]-=1/(1j*omega*L[n].value)
                if c!=0:
                    for n in range(c):
                        if((int(C[n].node_ini)==i+1 and int(C[n].node_fin)==j+1) or ((int(C[n].node_ini)==j+1 and int(C[n].node_fin)==i+1))):
                            matA[i][j]-=1j*omega*C[n].value

    for i in range(num_node):
        if vac!=0:
            for j in range(vac):
                if int(V_ac[j].node_ini)==i+1:
                    matA[i][j+num_node]=1
                    matA[j+num_node][i]=1
                elif int(V_ac[j].node_fin)==i+1:
                    matA[i][j+num_node]=-1
                    matA[j+num_node][i]=-1
        if vdc!=0:
            for j in range(vdc):
                if int(V[j].node_ini)==i+1:
                    matA[i][j+num_node]=1
                    matA[j+num_node][i]=1

                elif int(V[j].node_fin)==i+1:
                    matA[i][j+num_node]=-1
                    matA[j+num_node][i]=-1

    #Here we are adding in the values of the last column in the augmented matrix
    for i in range(num_node):
        if qac!=0:
            for j in range(qac):
                if i+1 == int(I_ac[j].node_fin):
                    matA[i][num_node+vac+vdc]+=float(I_ac[j].value)
        if qdc!=0:
            for j in range(qdc):
                if i+1 == int(I[j].node_fin):
                    matA[i][num_node+vac+vdc]+=float(I[j].value)
    if ac!=0:
        for i in range(vac):
            matA[i+num_node][num_node+vac+vdc]+=float(V_ac[i].value)
    if dc!=0:
        for i in range(vdc):
            matA[num_node+i][num_node+vac+vdc]+=float(V[i].value)


    N=num_node+vac+vdc #defining N for the gaussElim function
    ans=gaussElim(matA)
    # for i in range(num_node):
    #     print("V",i+1,"=",ans[i],"V")
    # for i in range(vac+vdc):
    #     print("I",i+1,"=",ans[i+num_node],"A")

Now we shall annotate this and optimize using Cython, using the same `type` and `for` loop optimization.

I have marked `type` optimization with `#t` and `for` loop optimization with `#f` as a comment, in the code.

In [8]:
%%cython --annotate
def spice_solver_opt(file):
    class resistor:
        def __init__(self,node_ini,node_fin,value):
            self.node_ini=node_ini
            self.node_fin=node_fin
            self.value=value
    class inductor:
        def __init__(self,node_ini,node_fin,value):
            self.node_ini=node_ini
            self.node_fin=node_fin
            self.value=value
    class capacitor:
        def __init__(self,node_ini,node_fin,value):
            self.node_ini=node_ini
            self.node_fin=node_fin
            self.value=value
    class voltage_source:
        def __init__(self,node_ini,node_fin,value):
            self.node_ini=node_ini
            self.node_fin=node_fin
            self.value=value
    class current_source:
        def __init__(self,node_ini,node_fin,value):
            self.node_ini=node_ini
            self.node_fin=node_fin
            self.value=value
    class current_source_ac:
        def __init__(self,node_ini,node_fin,value):
            self.node_ini=node_ini
            self.node_fin=node_fin
            self.value=value
    class voltage_source_ac:
        def __init__(self,node_ini,node_fin,value):
            self.node_ini=node_ini
            self.node_fin=node_fin
            self.value=value

    # Have initialised lists to hold objects of the components
    R=[0 for element in range(20)]
    C=[0 for element in range(20)]
    L=[0 for element in range(20)]
    V=[0 for element in range(20)]
    I=[0 for element in range(20)]
    V_ac=[0 for element in range(20)]
    I_ac=[0 for element in range(20)]
    cdef int r #t
    cdef int c
    cdef int w
    cdef int qac
    cdef int qdc
    cdef int vac
    cdef int vdc
    cdef int dc
    cdef int ac
    cdef int num_node
    cdef int f
    cdef int g
    #counter variables to count number of components and nodes
    r=c=w=qac=qdc=vac=vdc=dc=ac=num_node=f=0
    g=0
    file1 = open(file,'r') #We can check for other test cases by editing the path over here
    lines = file1.readlines()
    cdef float node3 #t
    cdef float node4
    for l in lines:
        #To find the lines between the start and end
        if l.strip() == ".end" or l.strip() == ".circuit":
            g=1
        if g==1:
            temp=l[0] #t
            if temp == 'R':
                #this changes the value of the list to an object of the component with the attributes and simultanouesly checks for n1 GND and changes them to integers
                R[r]=(resistor(l.split()[1].replace('n','').replace('GND','0'),l.split()[2].replace('n','').replace('GND','0'),float(l.split()[3])))
                num_node=max(int(R[r].node_ini),num_node,int(R[r].node_fin))
                r+=1
            elif temp =='V':
                if str(l.split()[3])=="ac":  #to check for ac source
                    V_ac[vac]=(voltage_source_ac(l.split()[1].replace('n','').replace('GND','0'),l.split()[2].replace('n','').replace('GND','0'),float(l.split()[4])))
                    num_node=max(int(V_ac[vac].node_ini),num_node,int(V_ac[vac].node_fin))
                    ac+=1
                    vac+=1
                elif str(l.split()[3]) == "dc":
                    V[vdc]=(voltage_source(l.split()[1].replace('n','').replace('GND','0'),l.split()[2].replace('n','').replace('GND','0'),float(l.split()[4])))
                    num_node=max(int(V[vdc].node_ini),num_node,int(V[vdc].node_fin))
                    dc+=1
                    vdc+=1
            elif temp=='I':
                if str(l.split()[3])=="dc":
                    I[qdc]=(current_source(l.split()[1].replace('n','').replace('GND','0'),l.split()[2].replace('n','').replace('GND','0'),float(l.split()[4])))
                    num_node=max(int(I[qdc].node_ini),num_node,int(I[qdc].node_fin))
                    dc+=1
                    qdc+=11
                elif str(l.split()[3])=="ac":
                    I_ac[qac]=(current_source_ac(l.split()[1].replace('n','').replace('GND','0'),l.split()[2].replace('n','').replace('GND','0'),float(l.split()[4])))
                    num_node=max(int(I_ac[qac].node_ini),num_node,int(I_ac[qac].node_fin))
                    ac+=1
                    qac+=1
            elif temp =='L':
                L[w]=(inductor(l.split()[1].replace('n','').replace('GND','0'),l.split()[2].replace('n','').replace('GND','0'),float(l.split()[3])))
                num_node=max(int(L[w].node_ini),num_node,int(L[w].node_fin))
                w+=1
            elif temp =='C':
                C[c]=(capacitor(l.split()[1].replace('n','').replace('GND','0'),l.split()[2].replace('n','').replace('GND','0'),float(l.split()[3])))
                num_node=max(int(C[c].node_ini),num_node,int(C[c].node_fin))
                c+=1
        if g==2:  #breaks after ".end" is read
            break
    checker = l[0:3]
    cdef float freq #t
    cdef float omega
    for l in lines:  #loop to find frequency of the AC source
        if checker=='.ac':
            freq=float(l.split()[2])
            omega=2*3.141592653*float(freq)
            f+=1
    file1.close()

    #to check for error states
    if (ac*dc!=0):
        print("Not possible to solve with AC and DC sources in one circuit")
        return None
    elif f>1:
        print("Not possible to solve circuit with two different AC frequencies")
        return None

    #the size of matrix would be n+m where n is number of nodes and m is number of independent voltage sources
    matA=[[0 for i in range(num_node+vac+vdc+1)]for element in range(num_node+vac+vdc)]

    #here we are first filling in the values of an nxn matrix within the larger matrix, with conductance values
    cdef int i #t
    cdef int j
    cdef int n
    i=0
    while i < num_node : #f
        j=0
        while j < num_node : #f
            if i==j:
                n=0
                while n < r: #f
                    if int(R[n].node_ini)==i+1 or int(R[n].node_fin)==i+1:
                        matA[i][i]+=1/(float(R[n].value))
                    n+=1
                if w!=0:
                    n=0
                    while n <w : #f
                        if int(L[n].node_ini)==i+1 or int(L[n].node_fin)==i+1:
                            matA[i][i]+=1/(1j*omega*L[n].value)
                        n+=1
                if c!=0:
                    n=0
                    while n< c: #f
                        if int(C[n].node_ini)==i+1 or int(C[n].node_fin)==i+1:
                            matA[i][i]+=1j*omega*C[n].value
                        n+=1
            else:
                n=0
                while n < r: #f
                    if((int(R[n].node_ini)==i+1 and int(R[n].node_fin)==j+1) or ((int(R[n].node_ini)==j+1 and int(R[n].node_fin)==i+1))):
                        matA[i][j]-=1/(float(R[n].value))
                    n+=1
                if w!=0:
                    n=0
                    while n < w: #f
                        if((int(L[n].node_ini)==i+1 and int(L[n].node_fin)==j+1) or ((int(L[n].node_ini)==j+1 and int(L[n].node_fin)==i+1))):
                            matA[i][j]-=1/(1j*omega*L[n].value)
                        n+=1
                if c!=0:
                    n=0
                    while n < c: #f
                        if((int(C[n].node_ini)==i+1 and int(C[n].node_fin)==j+1) or ((int(C[n].node_ini)==j+1 and int(C[n].node_fin)==i+1))):
                            matA[i][j]-=1j*omega*C[n].value
                        n+=1
            j+=1
        i+=1
    i=0
    while i < num_node: #f
        if vac!=0:
            j=0
            while j < vac: #f
                if int(V_ac[j].node_ini)==i+1:
                    matA[i][j+num_node]=1
                    matA[j+num_node][i]=1
                elif int(V_ac[j].node_fin)==i+1:
                    matA[i][j+num_node]=-1
                    matA[j+num_node][i]=-1
                j+=1
        if vdc!=0:
            j=0
            while j < vdc: #f
                if int(V[j].node_ini)==i+1:
                    matA[i][j+num_node]=1
                    matA[j+num_node][i]=1

                elif int(V[j].node_fin)==i+1:
                    matA[i][j+num_node]=-1
                    matA[j+num_node][i]=-1
                j+=1
        i+=1

    #Here we are adding in the values of the last column in the augmented matrix
    i=0
    while i< num_node: #f
        if qac!=0:
            j=0
            while j < qac :
                if i+1 == int(I_ac[j].node_fin):
                    matA[i][num_node+vac+vdc]+=float(I_ac[j].value)
                j+=1
        if qdc!=0:
            j=0
            while j < qdc: #f
                if i+1 == int(I[j].node_fin):
                    matA[i][num_node+vac+vdc]+=float(I[j].value)
                j+=1
        i+=1
    if ac!=0:
        i=0
        while i<vac: #f
            matA[i+num_node][num_node+vac+vdc]+=float(V_ac[i].value)
            i+=1
    if dc!=0:
        i=0
        while i < vdc: #f
            matA[num_node+i][num_node+vac+vdc]+=float(V[i].value)
            i+=1
    cdef int N #t
    N =num_node+vac+vdc #defining N for the gaussElim function
    def gaussElim(mat): 
        cdef int singular_flag #t
        singular_flag = Elim(mat)
        if singular_flag!=-1:
            print("It's a Singular Matrix")
            if(mat[singular_flag][N]):
                print("System is not consistent")
            else:
                print("It has infinitely many solutions")
            return
        return(back_sub(mat))

    def swap_row(mat,i,j):
        cdef int k #t
        k=0
        while k<N+1: #f
            temp=mat[i][k]
            mat[i][k]=mat[j][k]
            mat[j][k]=temp
            k+=1

    def Elim(mat):
        cdef int k #t
        cdef int i
        cdef float f
        k=0
        while k < N: #f
            im=k
            vm=abs(mat[im][k])
            i = k+1
            while i < N: #f
                if(abs(mat[i][k])>vm):
                    vm=abs(mat[i][k])
                    im=i
                i+=1
            if not mat[k][im]:
                return k
            if im!=k:
                swap_row(mat,k,im)
            i=k+1
            while i < N: #f
                f=mat[i][k]/mat[k][k]
                for j in range(k+1,N+1):
                    mat[i][j]-=mat[k][j]*f
                mat[i][k]=0
                i+=1
            k+=1
        return -1

    def back_sub(mat):
        x=[0 for element in range(N)]
        cdef int i #t
        i=N-1
        while i>-1: #f
            x[i]=mat[i][N]
            j=i+1
            while j < N: #f
                x[i]-=mat[i][j]*x[j]
                j+=1
            x[i]=(x[i]/mat[i][i])
            i-=1
        return(x)
    ans=gaussElim(matA)
    # for i in range(num_node):
    #     print("V",i+1,"=",ans[i],"V")
    # for i in range(vac+vdc):
    #     print("I",i+1,"=",ans[i+num_node],"A")
        

In [9]:
solve='ckt1.netlist'
%timeit spice_solver(solve)
%timeit spice_solver_opt(solve)

178 µs ± 11.6 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
125 µs ± 8.57 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [ ]:
solve='ckt3.netlist'
%timeit spice_solver(solve)
%timeit spice_solver_opt(solve)

235 µs ± 5.75 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [ ]:
solve='ckt4.netlist'
%timeit spice_solver(solve)
%timeit spice_solver_opt(solve)

In [ ]:
solve='ckt5.netlist'
%timeit spice_solver(solve)
%timeit spice_solver_opt(solve)

Here are my results after taking a few netlists as examples.

We can clearly see that the Cython version is faster although not as much as we'd like. I suspect this is because of class object declaration within the function itself. I tried to define it outside but ran into problems where Cython could not recognize the classes defined outside. I think if I could overcome that, Cython version would be much faster.